In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np
from itertools import combinations 

GYAccount_id="act_814959101969384"

list_account_id=[GYAccount_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

In [2]:
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,
                    AudienceDescription,
                    ACCESS_TOKEN,
                    accountid
                   ): # fb_schema=CustomAudience.Schema.email_hash
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    
    '''
    AudienceName: str, the name shown on FB
    AudienceDescription: str, the description shown on FB
    df: pd.Dataframe, input with the identifier col
    ACCESS_TOKEN: str, access to FB marketing api
    accountid: str, account to uplaod
    df_use_col_name: str, col name in the df
    fb_schema: available schema attribution in the CustomAudience.Schema class. Default for us CustomAudience.Schema.email_hash  
    https://github.com/facebook/facebook-python-business-sdk/blob/ac390147132101763a99a917bb59bdc8a374a7ce/facebook_business/adobjects/helpers/customaudiencemixin.py
    '''
    
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("New Audience created with FB_ID:",fbname)
    return fbname


def add_users_to_created_audience(fb_audience_id,df_input,dict_col_and_fbschema,is_raw_boolean,AudienceName,chunck_size=50000):
    audience = CustomAudience(fb_audience_id)
    count_uploaded=0
    
    # single columns
    if len(dict_col_and_fbschema)==1:
        print("single_key: \n",dict_col_and_fbschema)
        uploading_schema=dict_col_and_fbschema.values()[0]
        df_use_col_name=dict_col_and_fbschema.keys()[0]
        users = list(df_input[df_use_col_name])
        
        listlen = len(users)
        for i in range(int(np.ceil(listlen/chunck_size))):
            starti = i*chunck_size
            if (i+1)*chunck_size<listlen:
                endi = (i+1)*chunck_size
            else:
                endi = listlen
            list0 = users[starti:endi]

            count_uploaded+=len(list0)
            audience.add_users(uploading_schema, list0) #,is_raw=True
        
        
    else:
        print("multiple_key: \n",dict_col_and_fbschema)
        list_all_comb_cols_keys=[]
        for len_n in range(len(dict_col_and_fbschema)):  
            perm = combinations(dict_col_and_fbschema.keys(),len_n+1)
            for a in list(perm):
                list_all_comb_cols_keys.append(a)

        for comb in list_all_comb_cols_keys:
            selected_schema=[]
            remaining_cols=list(dict_col_and_fbschema.keys()).copy()
            df_select=df_input[dict_col_and_fbschema.keys()].copy()
            for col in list(comb):
                remaining_cols.remove(col)
                df_select=df_select[pd.notnull(df_select[col])]
                selected_schema.append(dict_col_and_fbschema[col])
                
            if len(remaining_cols)>0:
                for col in remaining_cols:
                    df_select=df_select[pd.isnull(df_select[col])]                
            
            selected_users=df_select[list(comb)].values.tolist()

            listlen = len(selected_users)
            for i in range(int(np.ceil(listlen/chunck_size))):
                starti = i*chunck_size
                if (i+1)*chunck_size<listlen:
                    endi = (i+1)*chunck_size
                else:
                    endi = listlen
                list0 = selected_users[starti:endi]

                count_uploaded+=len(list0)
                if len(selected_schema)==1:
                    audience.add_users(selected_schema, list0,is_raw=is_raw_boolean,)
                else:
                    audience.add_users(selected_schema, list0,is_raw=is_raw_boolean)

    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded,"\n")


In [3]:
import glob
folder_segs="./output_cleaned_audience_2020-11-06/"
list_files_to_upload=glob.glob(folder_segs+"*.csv")
list_files_to_upload.sort()
print(len(list_files_to_upload))
list_files_to_upload

40


['./output_cleaned_audience_2020-11-06/segment_Control_ServiceOnly_Balance_stores_Store_20200301_20200930.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Ecommerce_All_All.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Store_P_R1H.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Store_P_R2M.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Store_P_R3L.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Store_S_R1H.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Store_S_R2M.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_31_stores_Store_S_R3L.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_9_stores_Ecommerce_All_All.csv',
 './output_cleaned_audience_2020-11-06/segment_Control_TireAndService_9_stores_Store_P_R1H.cs

In [4]:
for file in list_files_to_upload:
    for fb_account_id in list_account_id:
        audience_name = os.path.basename(file).split(".")[0]
        description=os.path.basename(file).split(".")[0]+"_Juba_JL_"+str(datetime.datetime.now().date())
        df = pd.read_csv(file,dtype=str)
        print(fb_account_id,audience_name,df.shape)
        audience_name="aud_API_"+audience_name
        aud_id=creatfbaudience(AudienceName=audience_name,
                    AudienceDescription=description+"updated_on_%s"%str(datetime.datetime.now().date()),
                    ACCESS_TOKEN=jian_token,
                    accountid=fb_account_id,
                   )
        
        add_users_to_created_audience(fb_audience_id=aud_id,
                                      df_input=df,
                                      AudienceName=audience_name,
                                      dict_col_and_fbschema={"best_email_txt":CustomAudience.Schema.MultiKeySchema.email,
                                                             "best_phone_num":CustomAudience.Schema.MultiKeySchema.phone},
                                      chunck_size=50000,
                                      is_raw_boolean=True)
        
        

act_814959101969384 segment_Control_ServiceOnly_Balance_stores_Store_20200301_20200930 (72391, 11)
New Audience created with FB_ID: 6226404972506
multiple_key: 
 {'best_email_txt': 'EMAIL', 'best_phone_num': 'PHONE'}
2020-11-06 14:24:47.611943 aud_API_segment_Control_ServiceOnly_Balance_stores_Store_20200301_20200930 uploaded 71802 

act_814959101969384 segment_Control_TireAndService_31_stores_Ecommerce_All_All (19202, 5)
New Audience created with FB_ID: 6226404981906
multiple_key: 
 {'best_email_txt': 'EMAIL', 'best_phone_num': 'PHONE'}
2020-11-06 14:24:53.539484 aud_API_segment_Control_TireAndService_31_stores_Ecommerce_All_All uploaded 19194 

act_814959101969384 segment_Control_TireAndService_31_stores_Store_P_R1H (34784, 6)
New Audience created with FB_ID: 6226404998906
multiple_key: 
 {'best_email_txt': 'EMAIL', 'best_phone_num': 'PHONE'}
2020-11-06 14:24:57.640347 aud_API_segment_Control_TireAndService_31_stores_Store_P_R1H uploaded 34664 

act_814959101969384 segment_Control_Ti

2020-11-06 14:28:10.231878 aud_API_segment_Test_TireAndService_31_stores_Store_S_R2M uploaded 103263 

act_814959101969384 segment_Test_TireAndService_31_stores_Store_S_R3L (102560, 5)
New Audience created with FB_ID: 6226405315106
multiple_key: 
 {'best_email_txt': 'EMAIL', 'best_phone_num': 'PHONE'}
2020-11-06 14:28:17.240739 aud_API_segment_Test_TireAndService_31_stores_Store_S_R3L uploaded 100197 

act_814959101969384 segment_Test_TireAndService_9_stores_Ecommerce_All_All (4497, 5)
New Audience created with FB_ID: 6226405321906
multiple_key: 
 {'best_email_txt': 'EMAIL', 'best_phone_num': 'PHONE'}
2020-11-06 14:28:19.967958 aud_API_segment_Test_TireAndService_9_stores_Ecommerce_All_All uploaded 4496 

act_814959101969384 segment_Test_TireAndService_9_stores_Store_P_R1H (8648, 5)
New Audience created with FB_ID: 6226405323106
multiple_key: 
 {'best_email_txt': 'EMAIL', 'best_phone_num': 'PHONE'}
2020-11-06 14:28:23.990907 aud_API_segment_Test_TireAndService_9_stores_Store_P_R1H uplo